In [1]:
!pip install git+https://github.com/instadeepai/jumanji.git

  Cloning https://github.com/instadeepai/jumanji.git to /tmp/pip-req-build-m4e5mfex
  Running command git clone --filter=blob:none --quiet https://github.com/instadeepai/jumanji.git /tmp/pip-req-build-m4e5mfex
  Resolved https://github.com/instadeepai/jumanji.git to commit 185b9fadfafebfee17a3f861b6eed500dfdf52e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached jax-0.4.35-py3-none-any.whl.metadata (22 kB)
  Using cached toolz-1.0.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached dm_tree-0.1.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
  Using cached jaxlib-0.4.35-cp310-cp310-manylinux2014_x86_64.whl.metadata (983 bytes)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (

In [31]:
import jumanji
from jumanji.wrappers import AutoResetWrapper
import haiku as hk
import jax

env = jumanji.make("Game2048-v1")  # Create a Snake environment
env = AutoResetWrapper(env)  


In [32]:
batch_size = 3
rollout_length = 5
num_actions = env.action_spec.num_values

random_key = jax.random.PRNGKey(0)
key1, key2 = jax.random.split(random_key)

def step_fn(state, key):
  action = jax.random.randint(key=key, minval=0, maxval=num_actions, shape=())
  new_state, timestep = env.step(state, action)
  return new_state, timestep

def run_n_steps(state, key, n):
  random_keys = jax.random.split(key, n)
  state, rollout = jax.lax.scan(step_fn, state, random_keys)
  return rollout

# Instantiate a batch of environment states
keys = jax.random.split(key1, batch_size)
state, timestep = jax.vmap(env.reset)(keys)
print(state)
# Collect a batch of rollouts
keys = jax.random.split(key2, batch_size)
rollout = jax.vmap(run_n_steps, in_axes=(0, 0, None))(state, keys, rollout_length)

State(board=Array([[[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 0]],

       [[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]], dtype=int32), step_count=Array([0, 0, 0], dtype=int32), action_mask=Array([[False,  True,  True, False],
       [ True,  True,  True,  True],
       [False,  True,  True, False]], dtype=bool), score=Array([0., 0., 0.], dtype=float32), key=Array([[1079905802, 3703866665],
       [3121850452, 3194139882],
       [1289972510, 2727916387]], dtype=uint32))


In [49]:
print(rollout)

TimeStep(step_type=Array([[1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]], dtype=int8), reward=Array([[0., 0., 0., 4., 0.],
       [0., 4., 0., 0., 0.],
       [0., 4., 0., 4., 8.]], dtype=float32), discount=Array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]], dtype=float32), observation=Observation(board=Array([[[[1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]],

        [[0, 1, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 0, 0, 0]],

        [[1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 1, 0, 0]],

        [[1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 0],
         [2, 0, 0, 0]],

        [[1, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 1, 0],
         [2, 0, 0, 0]]],


       [[[0, 0, 1, 0],
         [0, 0, 0, 0],
         [0, 0, 1, 0],
         [0, 0, 0, 0]],

        [[0, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 0],
      

In [46]:
# print(env.action_spec.num_values)
import numpy as np

print(state.board.shape)

print(np.resize(state.board, (3, 16)))

# print(state.action_mask)

(3, 4, 4)
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [29]:
print(rollout)

TimeStep(step_type=Array([[1, 1, 1, 1, 1]], dtype=int8), reward=Array([[0., 0., 4., 0., 4.]], dtype=float32), discount=Array([[1., 1., 1., 1., 1.]], dtype=float32), observation=Observation(board=Array([[[[0, 0, 0, 0],
         [0, 0, 0, 1],
         [1, 0, 0, 0],
         [0, 0, 0, 0]],

        [[1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 0, 0, 1]],

        [[0, 0, 0, 1],
         [0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 1, 2]],

        [[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 1, 1],
         [0, 0, 1, 2]],

        [[0, 0, 2, 1],
         [0, 0, 1, 2],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]]], dtype=int32), action_mask=Array([[[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True, False,  True,  True],
        [ True,  True,  True,  True],
        [False, False,  True,  True]]], dtype=bool)), extras={'highest_tile': Array([[2, 2, 4, 4, 4]], dtype=int32)})
